# Monitoring Logging Coupes in Victoria

## Set up your workspace

In [ ]:
import datacube
import geopandas as gpd
import pandas as pd

from odc.algo import xr_geomedian
from datacube.utils.geometry import Geometry

from dea_tools.datahandling import load_ard
from dea_tools.spatial import xr_rasterize

from plotting_functions import plot_rgb_ndvi

# Change a pandas setting to view all columns and all rows of loaded data
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

## Load spatial file with coupes

In [ ]:
log_season_gdf = gpd.read_file("data/LOG_SEASON_FILTERED.gpkg")

## Initial exploration

In [ ]:
log_season_gdf.head()

## Analyse logging events from 2019

In [ ]:
# Restrict events to those that occurred during 2019

logging_events_2019 = log_season_gdf.loc[log_season_gdf.loc[:, "STARTYEAR_STRING"]=="2019", :].reset_index(drop=True)

In [ ]:
events_of_interest = {
    "Clearfelling": "14/770/507/0011/201920/00",
    "Regrowth retention harvesting": "08/286/505/0029/201920/00",
    "Variable retention 1": "16/686/510/0026/201920/00",
}

In [ ]:
logging_events_to_analyse = logging_events_2019.loc[logging_events_2019.loc[:, "LOGHISTID"].isin(list(events_of_interest.values())), :].reset_index(drop=True)

logging_events_to_analyse

### Extract satellite imagery

In [ ]:
dc = datacube.Datacube(app='Logging_analysis')

In [ ]:
time_range = ("2019-06-01", "2020-12-31")

products = ["s2a_ard_granule", "s2b_ard_granule"]
measurements = ["nbart_red", "nbart_green", "nbart_blue", "nbart_nir_1"]
resolution = (10, -10)
output_crs = "EPSG:3577"
min_gooddata=0.99

query = {
    "time": time_range,
    "measurements": measurements,
    "min_gooddata": min_gooddata,
    "resolution": resolution,
    "output_crs": "EPSG:3577",
    "group_by": "solar_day"
}

In [ ]:
# Time taken: 5 mins

event_imagery = {}

for event_type, event_id in events_of_interest.items(): #logging_events_to_analyse.iterrows():
    
    print(f"Analysing LOGHISTID {event_id}; Event type: {event_type}")
    
    event = logging_events_to_analyse.loc[logging_events_to_analyse.LOGHISTID==event_id]

    geometry = Geometry(geom=event.geometry.values[0], crs=logging_events_to_analyse.crs)
    query.update({"geopolygon": geometry})

    ds = load_ard(dc=dc,
                  products=["s2a_ard_granule", "s2b_ard_granule"],
                  **query)

    # Generate a polygon mask to keep only data within the polygon
    mask = xr_rasterize(event, ds)

    # Mask dataset to set pixels outside the polygon to `NaN`
    ds = ds.where(mask)

    # Group by 3 month intervals
    grouped = ds.resample(time='3MS')
    geomedian_grouped = grouped.map(xr_geomedian)
    
    # Calculate NDVI
    geomedian_grouped["NDVI"] = (geomedian_grouped.nbart_nir_1 - geomedian_grouped.nbart_red) / (geomedian_grouped.nbart_nir_1 + geomedian_grouped.nbart_red)
    
    event_imagery[event_type] = geomedian_grouped

In [ ]:
for event_type, event_data in event_imagery.items():
    plot_rgb_ndvi(event_data, event_type)

In [ ]:
import matplotlib.pyplot as plt

labels = []

fig, ax = plt.subplots(figsize=(12, 6))

fig.suptitle("Change in NDVI over time for different logging events", fontsize=16)

for event_type, event_data in event_imagery.items():
    event_data.NDVI.mean(dim=["x", "y"]).plot(add_legend=False, ax=ax)
    labels.append(event_type)

plt.legend(labels, ncol=1)
plt.show()
